# Introdução

## Importação das bibliotecas necessarias

In [1]:
%matplotlib inline
import requests
import pandas as pd
import os
import json
import sys

## Funções auxiliares

In [2]:
def return_sql_query(query_string):
    ret = {
        "token": {
            "token": token
        },
        "sql": {
            "sql": {
                "query":query_string
            }
        }
    }
    return ret

def json_to_df(resp):
    columns = [col['name'] for col in resp.json()['columns']]
    values = resp.json()['rows']  
    return pd.DataFrame(values, columns=columns)

def rows_and_columns_to_df(columns, rows):
    columns_array = [col['name'] for col in columns]
    return pd.DataFrame(rows, columns=columns_array)

def fetch_all_data_from_response(res, initial_records_rows):
    while 'cursor' in res.json():
        res_next = {
            "token": {
                "token": token
            },
            "sql": {
                "sql": {
                    "cursor": res.json()['cursor']
                }
            }
        }
        res = requests.post(os.path.join(url_base,'sql_query'), json=res_next)
        initial_records_rows+=res.json()['rows']
        sys.stdout.write('.')
        sys.stdout.flush()
    print('(y)')
    return initial_records_rows

## Dados de acesso a api

In [3]:
from IPython.display import clear_output
import getpass

token = getpass.getpass('Digite o seu token para acesso a API:')

clear_output()

In [7]:
url_base = "https://bigdata-api.fiocruz.br"

## Bases de dados disponiveis na API atualizadas

In [8]:
params = {
          "token": token
        }
resp = requests.post(url_base+ '/show_tables', json=params)
print(resp.json())

{'databases': ['datasus-sim', 'datasus-sinasc', 'datasus-sih']}


## Busca dos campos existentes na base SIM

In [9]:
%time
sql = {
          "token": {
            "token": token
          },
          "sql": {
            "sql": {"query":'describe "datasus-sim"'}
          }
        }
resp = requests.post(os.path.join(url_base,'sql_query'), json=sql)
#resp.content
fields = [x[0] for x in resp.json()['rows']]
print(fields)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 7.63 µs
['ACIDTRAB', 'ALTCAUSA', 'ASSISTMED', 'ATESTADO', 'ATESTANTE', 'CAUSABAS', 'CAUSABAS_O', 'CAUSAMAT', 'CB_PRE', 'CIRCOBITO', 'CIRURGIA', 'CODBAIOCOR', 'CODBAIRES', 'CODCART', 'CODESTAB', 'CODIFICADO', 'CODINST', 'CODMUNCART', 'CODMUNNATU', 'CODMUNOCOR', 'CODMUNRES', 'COMUNSVOIM', 'CONTADOR', 'CRM', 'DIFDATA', 'DTATESTADO', 'DTCADASTRO', 'DTCADINF', 'DTCADINV', 'DTCONCASO', 'DTCONINV', 'DTINVESTIG', 'DTNASC', 'DTOBITO', 'DTRECEBIM', 'DTRECORIG', 'DTRECORIGA', 'DTREGCART', 'ESC', 'ESC2010', 'ESCFALAGR1', 'ESCMAE', 'ESCMAE2010', 'ESCMAEAGR1', 'ESTABDESCR', 'ESTCIV', 'EXAME', 'EXPDIFDATA', 'FONTE', 'FONTEINV', 'FONTES', 'FONTESINF', 'GESTACAO', 'GRAVIDEZ', 'HORAOBITO', 'IDADE', 'IDADEMAE', 'LINHAA', 'LINHAB', 'LINHAC', 'LINHAD', 'LINHAII', 'LOCOCOR', 'MORTEPARTO', 'NATURAL', 'NECROPSIA', 'NUDIASINF', 'NUDIASOBCO', 'NUDIASOBIN', 'NUMERODN', 'NUMERODO', 'NUMERODV', 'NUMEROLOTE', 'NUMREGCART', 'NUMSUS', 'OBITOGRAV', 'OBITOPARTO', 

## Busca dos campos existentes na base SINASC

In [10]:
%time
sql = {
          "token": {
            "token": token
          },
          "sql": {
            "sql": {"query":'describe "datasus-sinasc"'}
          }
        }
resp = requests.post(os.path.join(url_base,'sql_query'), json=sql)
#resp.content
fields = [x[0] for x in resp.json()['rows']]
print(fields)

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 9.54 µs
['APGAR1', 'APGAR5', 'CODANOMAL', 'CODBAINASC', 'CODBAIRES', 'CODCART', 'CODESTAB', 'CODINST', 'CODMUNCART', 'CODMUNNASC', 'CODMUNNATU', 'CODMUNRES', 'CODOCUPMAE', 'CODPAISRES', 'CODUFNATU', 'CONSPRENAT', 'CONSULTAS', 'CONTADOR', 'DIFDATA', 'DTCADASTRO', 'DTDECLARAC', 'DTNASC', 'DTNASCMAE', 'DTRECEBIM', 'DTRECORIG', 'DTRECORIGA', 'DTREGCART', 'DTULTMENST', 'ESCMAE', 'ESCMAE2010', 'ESCMAEAGR1', 'ESTCIVMAE', 'GESTACAO', 'GRAVIDEZ', 'HORANASC', 'IDADEMAE', 'IDADEPAI', 'IDANOMAL', 'KOTELCHUCK', 'LOCNASC', 'MESPRENAT', 'NATURALMAE', 'NOVO', 'NUMERODN', 'NUMERODV', 'NUMEROLOTE', 'NUMREGCART', 'ORIGEM', 'PARIDADE', 'PARTO', 'PESO', 'PREFIXODN', 'QTDFILMORT', 'QTDFILVIVO', 'QTDGESTANT', 'QTDPARTCES', 'QTDPARTNOR', 'RACACOR', 'RACACORMAE', 'RACACORN', 'RACACOR_RN', 'SEMAGESTAC', 'SERIESCMAE', 'SEXO', 'STCESPARTO', 'STDNEPIDEM', 'STDNNOVA', 'STTRABPART', 'TPAPRESENT', 'TPDOCRESP', 'TPFUNCRESP', 'TPMETESTIM', 'TPNASCASSI', 'TPROBSON'

# Taxa de fecundidade total (SINASC + Tabnet)

A ideia inicial era explorar os dados de todo o Brasil do período de 2012 a 2019.
Como a tabela SIH-RD tratada e filtrada que temos é do período de 2018 até 2019 e apenas do Estado de São Paulo, iremos utilizar o mesmo subconjunto.

In [11]:
%time
sql_t_nasc_vivos = return_sql_query("SELECT CODMUNRES, ano_nasc as Ano, count(*) as Qtd_Total  \
                    FROM \"datasus-sinasc\" \
                    WHERE ano_nasc>=2012 AND \
                    ano_nasc<=2019 AND \
                    IDADEMAE>=10 AND \
                    IDADEMAE<=49 \
                    GROUP BY CODMUNRES, ano_nasc")
resp_sql_t_nasc_vivos = requests.post(os.path.join(url_base,'sql_query'), json=sql_t_nasc_vivos)
record_columns=resp_sql_t_nasc_vivos.json()['columns']
record_rows=resp_sql_t_nasc_vivos.json()['rows']

fetch_all_data_from_response(resp_sql_t_nasc_vivos, record_rows)
print("Record rows length : "+str(len(record_rows)))

df_t_nasc_vivos = rows_and_columns_to_df(record_columns, record_rows)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.58 µs
.............................................(y)
Record rows length : 44680


In [12]:
%time
# fonte do dado populacional
# http://tabnet.datasus.gov.br/cgi/deftohtm.exe?popsvs/cnv/popbr.def
df_pop_fem_10_49 = pd.read_csv('./tabelas_tabnet/pop_por_municipio_limpa.csv', ';')

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.82 µs


In [13]:
df_pop_fem_10_49.head()

,Municipio,2012,2013,2014,2015,2016,2017,2018,2019
0,110001,24516,24293,24066,23835,23614,23392,23167,22945
1,110037,13141,13155,13170,13184,13199,13213,13227,13241
2,110040,18388,18819,19258,19705,20134,20563,20999,21428
3,110034,16621,16308,15989,15664,15352,15040,14722,14411
4,110002,95836,97540,99278,101048,102744,104441,106168,107863


In [14]:
df_t_nasc_vivos.head()

,CODMUNRES,Ano,Qtd_Total
0,110000,2012,2
1,110000,2013,16
2,110000,2014,4
3,110000,2015,4
4,110000,2016,2


In [15]:
years = []
for col in df_pop_fem_10_49:
    if col != 'Municipio':
        years.append(col)

all_entries = []
for index, row in df_pop_fem_10_49.iterrows():
    row_array = row.values.tolist()
    for i in range(len(row_array)):
        if i != 0:
            pop = row_array[i]
            if pop == '-':
                pop = row_array[i+1]
            all_entries.append([row_array[0], years[i-1], int(pop)])

            
df_pop_fem_10_49_by_codmunres = pd.DataFrame(all_entries, columns=['CODMUNRES', 'Ano', 'Pop'])

df_pop_fem_10_49_by_codmunres.head()

,CODMUNRES,Ano,Pop
0,110001,2012,24516
1,110001,2013,24293
2,110001,2014,24066
3,110001,2015,23835
4,110001,2016,23614


In [16]:
df_t_nasc_vivos['Ano'] = df_t_nasc_vivos['Ano'].astype(int)
df_pop_fem_10_49_by_codmunres['CODMUNRES'] = df_pop_fem_10_49_by_codmunres['CODMUNRES'].astype(str)
df_pop_fem_10_49_by_codmunres['Ano'] = df_pop_fem_10_49_by_codmunres['Ano'].astype(int)
df_pop_fem_10_49_by_codmunres['Pop'] = df_pop_fem_10_49_by_codmunres['Pop'].astype(int)
df_merge = pd.merge(df_t_nasc_vivos, df_pop_fem_10_49_by_codmunres, how='right', on=['CODMUNRES','Ano'])
df_merge = df_merge.fillna(0)
df_merge['Ratio'] = df_merge['Qtd_Total']/df_merge['Pop']
df_merge.head()

,CODMUNRES,Ano,Qtd_Total,Pop,Ratio
0,110001,2012,386.0,24516,0.015745
1,110001,2013,382.0,24293,0.015725
2,110001,2014,341.0,24066,0.014169
3,110001,2015,373.0,23835,0.015649
4,110001,2016,326.0,23614,0.013805


In [17]:
%time
df_merge.to_csv('./csv_files/QR-271-taxa_fecundidade_total.csv',index=False)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.78 µs
